# Pre-processing 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import pycaret

In [2]:
#load dataframe
#parse datetime column
df=pd.read_csv('state_cov_weather2.csv', parse_dates=[0])
df.set_index('date', inplace= True)

In [3]:
df.head()

,VT_Avg_Temp(F),VT_PRCP(mm),ME_Avg_Temp(F),ME_PRCP(mm),CT_Avg_Temp(F),CT_PRCP(mm),MA_Avg_Temp(F),MA_PRCP(mm),CT_Conf_Cases,VT_Conf_Cases,ME_Conf_Cases,MA_Conf_Cases,weekday
date,,,,,,,,,,,,,
2020-03-01,19.58,0.3,14.18,0.3,26.24,0.0,26.42,0.0,0.0,0.0,0.0,1.0,6
2020-03-02,32.00,0.0,17.06,4.6,37.58,0.0,36.50,0.0,0.0,0.0,0.0,1.0,0
2020-03-03,44.06,2.8,39.56,0.0,51.26,3.8,55.94,0.0,0.0,0.0,0.0,1.0,1
2020-03-04,41.54,0.0,32.00,6.6,48.02,0.0,46.94,2.5,0.0,0.0,0.0,2.0,2
2020-03-05,37.04,0.0,30.02,0.0,43.16,0.0,42.98,0.0,0.0,0.0,0.0,8.0,3


In [4]:
df_vt=df.drop(columns=['ME_Avg_Temp(F)', 'ME_PRCP(mm)', 'ME_Conf_Cases', 'CT_Avg_Temp(F)', 'CT_PRCP(mm)', 'CT_Conf_Cases', 'MA_Avg_Temp(F)', 'MA_PRCP(mm)', 'MA_Conf_Cases','weekday'])

In [5]:
df_me=df.drop(columns=['VT_Avg_Temp(F)', 'VT_PRCP(mm)', 'VT_Conf_Cases', 'CT_Avg_Temp(F)', 'CT_PRCP(mm)', 'CT_Conf_Cases', 'MA_Avg_Temp(F)', 'MA_PRCP(mm)', 'MA_Conf_Cases','weekday'])

In [6]:
df_ma=df.drop(columns=['ME_Avg_Temp(F)', 'ME_PRCP(mm)', 'ME_Conf_Cases', 'CT_Avg_Temp(F)', 'CT_PRCP(mm)', 'CT_Conf_Cases', 'VT_Avg_Temp(F)', 'VT_PRCP(mm)', 'VT_Conf_Cases','weekday'])

In [7]:
df_ct= df.drop(columns=['ME_Avg_Temp(F)', 'ME_PRCP(mm)', 'ME_Conf_Cases', 'VT_Avg_Temp(F)', 'VT_PRCP(mm)', 'VT_Conf_Cases', 'MA_Avg_Temp(F)', 'MA_PRCP(mm)', 'MA_Conf_Cases','weekday'])

In [38]:
#Let's take a look at the distribution of case counts by state
#with sns.axes_style("white"):
    #fig, axes = plt.subplots(2,2, figsize=(15,12))
    #fig.suptitle('Distribution of new cases counts per day per states', fontsize=18)
    #axes[0,0].set_title('Massachusetts', fontsize=14)
    #axes[0,1].set_title('Connecticut', fontsize =14)
    #axes[1,0].set_title('Vermont', fontsize= 14)
    #axes[1,1].set_title('Maine', fontsize= 14)
    # Massachusetts
    #sns.histplot(ax=axes[0, 0], x=df_ma.MA_Conf_Cases, bins=30)
    # Connecticut
    #sns.histplot(ax=axes[0, 1], x=df_ct.CT_Conf_Cases, bins=30)
    # Vermont
    #sns.histplot(ax=axes[1, 0], x=df_vt.VT_Conf_Cases,bins=30)
    # Maine
    #sns.histplot(ax=axes[1, 1], x=df_me.ME_Conf_Cases, bins=30)

Now all of the data appears stationary and we have no more `NaN` values in any of the datasets.

***

In [9]:
# Make duplicates of datasets for processing with Pycaret:
df_vt3 = df_vt.reset_index()
df_ct3 = df_ct.reset_index()
df_me3 = df_me.reset_index()
df_ma3 = df_ma.reset_index()

In [10]:
df_list = [df_ct3, df_ma3, df_me3, df_vt3]

In [11]:
# Add 'day_of_week' and 'day_of_year' columns to each dataframe
for df in df_list:
    df['day_of_week'] = [i.dayofweek for i in df['date']]
    df['day_of_year'] = [i.dayofyear for i in df['date']]
    df['Year'] =[i.year for i in df['date']]
    df['Month'] = [i.month for i in df['date']]
    df['Day'] = [i.day for i in df['date']]

# US Census Bureau state populations
* Massachusetts: 7,029,917 
* Vermont: 643,077	
* Connecticut: 3,605,944	
* Maine: 1,362,359

Source: [U.S. Census Bureau](https://www.census.gov/quickfacts/fact/table/VT,CT,MA,ME/PST045219) April 1, 2020 Census.

Because some states are very densely populated, while others are much more sparsely populated, I'll calculate confirmed Covid cases as a percentage of overall population. 

In order to do this, I'll divide by the appropriate state's total population (as per the April 1, 2020 U.S. Census Bureau), and multiply by 100,000 for ten thousandths of a percent. 

In [12]:
df_ma3['new_case_percent_pop*'] = (df_ma3['MA_Conf_Cases'] / 70.29917)

In [13]:
df_ct3['new_case_percent_pop*'] = (df_ct3['CT_Conf_Cases'] / 36.05944)

In [14]:
df_me3['new_case_percent_pop*'] = (df_me3['ME_Conf_Cases'] / 13.62359)

In [15]:
df_vt3['new_case_percent_pop*'] = (df_vt3['VT_Conf_Cases'] / 6.43077)

Try the model using new cases as ten thousandths of a percent of population, rather than count.

## Using the mean as a baseline prediction model

#### Massachusetts

In [16]:
# Create features
X_ma, y_ma = df_ma3.drop('MA_Conf_Cases', axis =1), df_ma3.MA_Conf_Cases
# Make test and training split
X_train_ma, X_test_ma, y_train_ma, y_test_ma = train_test_split(X_ma, y_ma, test_size = 0.2, random_state=42)
dummy_mean_ma = DummyRegressor(strategy='mean')
# "Train" dummy regressor
dummy_mean_ma.fit(X_train_ma, y_train_ma)
# Get R-squared score
score_mean_ma = dummy_mean_ma.score(X_test_ma, y_test_ma) 
print("The R2 score of using the mean to predict Massachusetts' COVID19 cases is:", score_mean_ma)

The R2 score of using the mean to predict Massachusetts' COVID19 cases is: -0.053284810131488225


#### Connecticut

In [17]:
# Create features
X_ct, y_ct = df_ct3.drop('CT_Conf_Cases', axis =1), df_ct3.CT_Conf_Cases
# Make test and training split
X_train_ct, X_test_ct, y_train_ct, y_test_ct = train_test_split(X_ct, y_ct, test_size=0.2, random_state=42)
# Create a dummy regressor
dummy_mean_ct = DummyRegressor(strategy='mean')
# "Train" dummy regressor
dummy_mean_ct.fit(X_train_ct, y_train_ct)
# Get R-squared score
score_mean_ct = dummy_mean_ct.score(X_test_ct, y_test_ct) 
print("The R2 score of using the mean to predict Connecticut's COVID19 cases is:", score_mean_ct)

The R2 score of using the mean to predict Connecticut's COVID19 cases is: -0.006702217631290974


#### Vermont

In [18]:
# Create features
X_vt, y_vt = df_vt3.drop('VT_Conf_Cases', axis = 1), df_vt3.VT_Conf_Cases
# Make test and training split
X_train_vt, X_test_vt, y_train_vt, y_test_vt = train_test_split(X_vt, y_vt, test_size = 0.2,random_state=42)
# Create a dummy regressor
dummy_mean_vt = DummyRegressor(strategy='mean')
# "Train" dummy regressor
dummy_mean_vt.fit(X_train_vt, y_train_vt)
# Get R-squared score
score_mean_vt = dummy_mean_vt.score(X_test_vt, y_test_vt) 
print("The R2 score of using the mean to predict Vermont's COVID19 cases is:", score_mean_vt)

The R2 score of using the mean to predict Vermont's COVID19 cases is: -0.013915984644440993


#### Maine

In [19]:
# Create features
X_me, y_me = df_me3.drop('ME_Conf_Cases', axis =1), df_me3.ME_Conf_Cases
# Make test and training split
X_train_me, X_test_me, y_train_me, y_test_me = train_test_split(X_me, y_me, test_size=0.2,random_state=42)
# Create a dummy regressor
dummy_mean_me = DummyRegressor(strategy='mean')
# "Train" dummy regressor
dummy_mean_me.fit(X_train_me, y_train_me)
# Get R-squared score
score_mean_me = dummy_mean_me.score(X_test_me, y_test_me) 
print("The R2 score of using the mean to predict Maine's COVID19 cases is:", score_mean_me)

The R2 score of using the mean to predict Maine's COVID19 cases is: -0.10540068459291052


## Merge state dataframes back into one dataframe

In order to do this, first we'll make a separate column in each state's dataframe indicating state_id. With this new identifier column, we'll no longer need state-specific identifiers in the column names themselves, and we can update each state's column names to mirror each other in preparation for the merge. Before we do this, we can go ahead and drop the PRCP columns for each state, as precipitation has proved to have little correlation with (and even less predictive power of) COVID19 cases.

In [20]:
# Create identifying state_id columns for each state df
df_ma3['state_id'] = 'MA'
df_ct3['state_id'] = 'CT'
df_vt3['state_id'] = 'VT'
df_me3['state_id'] = 'ME'

In [21]:
# Prepare df_ma3 for merge by updating column names
df_ma3.drop('MA_PRCP(mm)', axis=1, inplace=True)
ma_cols = {'MA_Avg_Temp(F)': 'Avg_Temp(F)',
           'MA_Conf_Cases' : 'Conf_Cases'}
df_ma3.rename(columns=ma_cols, inplace=True)

In [22]:
# Prepare df_ct3 for merge by updating column names
df_ct3.drop('CT_PRCP(mm)', axis=1, inplace=True)
ct_cols = {'CT_Avg_Temp(F)': 'Avg_Temp(F)',
           'CT_Conf_Cases' : 'Conf_Cases'}
df_ct3.rename(columns=ct_cols, inplace=True)

In [23]:
# Prepare df_vt3 for merge by updating column names
df_vt3.drop('VT_PRCP(mm)', axis=1, inplace=True)
vt_cols = {'VT_Avg_Temp(F)': 'Avg_Temp(F)',
           'VT_Conf_Cases' : 'Conf_Cases'}
df_vt3.rename(columns=vt_cols, inplace=True)

In [24]:
# Prepare df_me3 for merge by updating column names
df_me3.drop('ME_PRCP(mm)', axis=1, inplace=True)
me_cols = {'ME_Avg_Temp(F)': 'Avg_Temp(F)',
           'ME_Conf_Cases' : 'Conf_Cases'}
df_me3.rename(columns=me_cols, inplace=True)

In [30]:
df = pd.concat([df_ma3, df_me3, df_vt3, df_ct3], axis=0).set_index('date').reset_index()

In [31]:
df

,date,Avg_Temp(F),Conf_Cases,day_of_week,day_of_year,Year,Month,Day,new_case_percent_pop*,state_id
0,2020-03-01,26.42,1.0,6,61,2020,3,1,0.014225,MA
1,2020-03-02,36.50,1.0,0,62,2020,3,2,0.014225,MA
2,2020-03-03,55.94,1.0,1,63,2020,3,3,0.014225,MA
3,2020-03-04,46.94,2.0,2,64,2020,3,4,0.028450,MA
4,2020-03-05,42.98,8.0,3,65,2020,3,5,0.113799,MA
...,...,...,...,...,...,...,...,...,...,...
2219,2021-09-08,70.16,935.0,2,251,2021,9,8,25.929410,CT
2220,2021-09-09,71.42,626.0,3,252,2021,9,9,17.360225,CT
2221,2021-09-10,66.92,625.0,4,253,2021,9,10,17.332493,CT
2222,2021-09-11,63.14,0.0,5,254,2021,9,11,0.000000,CT


In [41]:
import plotly.express as px
for i in df['state_id'].unique():
    subset = df[df['state_id'] == i]
    subset.loc[:,'moving_average'] = subset['new_case_percent_pop*'].rolling(7).mean()
    fig = px.line(subset, x="date", y=["new_case_percent_pop*","moving_average"], title = (i + ' weekly rolling average'), template = 'plotly_dark')
    fig.show()

In [43]:
import plotly.express as px
for i in df['state_id'].unique():
    subset = df[df['state_id'] == i]
    subset.loc[:,'moving_average'] = subset['new_case_percent_pop*'].rolling(14).mean()
    fig = px.line(subset, x="date", y=["new_case_percent_pop*","moving_average"], title = (i + ' weekly rolling average'), template = 'plotly_dark')
    fig.show()

/Users/abigailmorgan/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/abigailmorgan/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Create a baseline model for optimization

In [ ]:
from pycaret.regression import *
all_results=[]
df_subset = df_ma3

# initialize setup from pycaret.regression
s = setup(df_subset, target = 'new_case_percent_pop*', train_size = 0.8,
            data_split_shuffle = True, fold = 3,
            ignore_features = ['date', 'MA_Conf_Cases', 'MA_PRCP(mm)'],
            numeric_features = ['day_of_year', 'Year', 'MA_Avg_Temp(F)'],
            categorical_features = ['Month', 'day_of_week'],
            silent = True, verbose = False, session_id = 123,
            normalize=False)
    
# compare all models and select best one based on MAE
best_model = compare_models(sort = 'R2', verbose=False)
    
# capture the compare result grid and store best model in list
p = pull().iloc[0:1]
p['time_series'] = str("MA")
all_results.append(p)
    
# finalize model i.e. fit on entire data including test set
f = finalize_model(best_model)
best_model

In [ ]:
concat_results = pd.concat(all_results,axis=0)
concat_results.head()

In [ ]:
all_results=[]
df_subset = df_ct3

# initialize setup from pycaret.regression
s = setup(df_subset, target = 'new_case_percent_pop*', train_size = 0.8,
            data_split_shuffle = True, fold = 3,
            ignore_features = ['date', 'CT_Conf_Cases', 'CT_PRCP(mm)'],
            numeric_features = ['day_of_year', 'Year', 'CT_Avg_Temp(F)'],
            categorical_features = ['Month', 'day_of_week'],
            silent = True, verbose = False, session_id = 123,
            normalize=True)
    
# compare all models and select best one based on MAE
best_model = compare_models(sort = 'R2', verbose=False)
    
# capture the compare result grid and store best model in list
p = pull().iloc[0:1]
p['time_series'] = str("CT")
all_results.append(p)
    
# finalize model i.e. fit on entire data including test set
f = finalize_model(best_model)
best_model

In [ ]:
concat_results = pd.concat([pd.concat(all_results, axis=0), concat_results], axis=0)
concat_results

In [ ]:
all_results=[]
df_subset = df_vt3

# initialize setup from pycaret.regression
s = setup(df_subset, target = 'new_case_percent_pop*', train_size = 0.8,
            data_split_shuffle = True, fold = 3,
            ignore_features = ['date', 'VT_Conf_Cases', 'VT_PRCP(mm)'],
            numeric_features = ['day_of_year', 'Year', 'VT_Avg_Temp(F)'],
            categorical_features = ['Month', 'day_of_week'],
            silent = True, verbose = False, session_id = 123,
            normalize=True)
    
# compare all models and select best one based on MAE
best_model = compare_models(sort = 'R2', verbose=False)
    
# capture the compare result grid and store best model in list
p = pull().iloc[0:1]
p['time_series'] = str("VT")
all_results.append(p)
    
# finalize model i.e. fit on entire data including test set
f = finalize_model(best_model)
best_model

In [ ]:
concat_results = pd.concat([pd.concat(all_results, axis=0), concat_results], axis=0)
concat_results

In [ ]:
all_results=[]
df_subset = df_me3

# initialize setup from pycaret.regression
s = setup(df_subset, target = 'new_case_percent_pop*', train_size = 0.8,
            data_split_shuffle = True, fold = 3,
            ignore_features = ['date', 'ME_Conf_Cases', 'ME_PRCP(mm)'],
            numeric_features = ['day_of_year', 'Year','ME_Avg_Temp(F)'],
            categorical_features = ['Month', 'day_of_week'],
            silent = True, verbose = False, session_id = 123,
            normalize=True)
    
# compare all models and select best one based on MAE
best_model = compare_models(sort = 'R2', verbose=False)
    
# capture the compare result grid and store best model in list
p = pull().iloc[0:1]
p['time_series'] = str("ME")
all_results.append(p)
    
# finalize model i.e. fit on entire data including test set
f = finalize_model(best_model)
best_model

In [ ]:
concat_results = pd.concat([pd.concat(all_results, axis=0), concat_results], axis=0)
concat_results

***